In [1]:
# generate a matrix of different options, run GAMs a ton of times
# we are just using the current GAMs input files placed in data/save_for_GAMs

from multiprocessing import Pool
from itertools import product
import numpy as np
import pandas as pd
import os
import shutil
import sys
sys.path.insert(0, '../functions/')
import interface_GAMS as iG

t_half_life_deg = 300
stable_flags = { # these do not change gene by gene
    # overall
    'only_check_KdRNAPCrp' : False, # if True, quit out of code after generating KdRNAPCrp, done to see if it is generating valid values through sanity check plots
    'save_results' : True, # saves resulting figures and cAct/cInh values of the previous run to the save_results_run folder
    'include_Amy_samples' : True, # append on Amy's stationary phase samples to analysis
    
    # GAMs
    'supress_output' : True, # don't show output from GAMs
    'use_greedy' : True, # use the greedy algo values (if False, uses the results of the GA)
    'run_on_all' : False, # run on all genes that are in the saved output folder
    'limit_samples' : ['b1101', 'b1817', 'b1818', 'b1819'], # if run_on_all is False, limit to these samples (or which of them are available)
    'delete_old' : True,
    'run_seperate' : False, # run cActivator and cInhibitor solvers seperately
    # input constants for GAMs (all get logged inside GAMs so pass in un-logged)
    'act_TF_conc_lo' : 1e-10,
    'act_TF_conc_up' : 1e-5,
    'act_Kd_lo' : 1e-10,
    'act_Kd_up' : 1e-6,
    'inh_TF_conc_lo' : 1e-10,
    'inh_TF_conc_up' : 1e-5,
    'inh_Kd_lo' : 1e-10,
    'inh_Kd_up' : 1e-6,
    # objective function weightings
    'weight_act_obj1' : 1,
    'weight_inh_obj1' : 1,
    'weight_act_obj2' : 0,
    'weight_inh_obj2' : 0,
    'weight_mRNA_match' : .1,
    'weight_act_corr' : 0.00000000000000001,
    'weight_inh_corr' : 0.00000000000000001,
    
    
    # misc
    'eq_str' : 'Eq(mRNARatio,((cActivator*KdRNAP + KdRNAPCrp)*(KdRNAP + RNAP + \
            KeqOpening*RNAP))/((1 + cActivator + cInhibitor)*KdRNAP*KdRNAPCrp + \
            cActivator*KdRNAP*(1 + KeqOpening)*RNAP + KdRNAPCrp*(1 + \
            KeqOpening)*RNAP))',
    
    # cell_constants'
    'cell_constants_RNAP': 10**-6,
    'cell_constants_mRNA_total': 1800, # Total mRNA/cell from https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3554401
    'cell_constants_cell_volume': 10**-15, # Liters from https://bionumbers.hms.harvard.edu/bionumber.aspx?id=100004&ver=19
    'cell_constants_kDeg': np.log(2)/t_half_life_deg, # Rate of degradation
    'cell_constants_promoterConcVal': 10**-9, # Promoter concentration
}

flags_df = pd.read_csv('../data/saved_flags.csv', index_col = 0)

In [2]:
# generate large list of sets of stable_flags with varying constants to run on
n_range = 3
min_val = 0.1
max_val = 1
lists = []
paras = ['act_TF_conc_lo', 'act_TF_conc_up', 'act_Kd_lo', 'act_Kd_up', 'weight_act_obj1', 'weight_mRNA_match', 'weight_act_corr']
for para in paras:
    lists.append(np.linspace(min_val*stable_flags[para], max_val*stable_flags[para], n_range))
combos = product(*lists)

cell_constants = {}
cell_constants.update({
    'RNAP' : stable_flags['cell_constants_RNAP'],
    'mRNA_total' : stable_flags['cell_constants_mRNA_total'],
    'cell_volume' : stable_flags['cell_constants_cell_volume'],
    'kDeg' : stable_flags['cell_constants_kDeg'],
    'promoterConcVal' : stable_flags['cell_constants_promoterConcVal'],
})

i = 0
inputs = []
for combo in list(combos):
    para_dict = {
        'act_TF_conc_lo' : combo[0],
        'act_TF_conc_up' : combo[1],
        'act_Kd_lo' : combo[2],
        'act_Kd_up' : combo[3],
        'inh_TF_conc_lo' : combo[0],
        'inh_TF_conc_up' : combo[1],
        'inh_Kd_lo' : combo[2],
        'inh_Kd_up' : combo[3],

        'weight_act_obj1' : combo[4],
        'weight_inh_obj1' : combo[4],
        'weight_act_obj2' : 0,
        'weight_inh_obj2' : 0,
        'weight_mRNA_match' : combo[5],
        'weight_act_corr' : combo[6],
        'weight_inh_corr' : combo[6],
    }
    GAMs_run_dir = '../GAMs/optimization_runs/run_'+str(i)
    inputs.append((flags_df, stable_flags, 'b3357', 'b1594', cell_constants, GAMs_run_dir, para_dict))
    i += 1

In [ ]:
# run GAMs
def mask_func(inp0, inp1, inp2, inp3, inp4, inp5, inp6):
    if os.path.exists(inp5):
        shutil.rmtree(inp5, ignore_errors = True)
    os.mkdir(inp5)
    os.mkdir(inp5+'/input_files')
    os.mkdir(inp5+'/output_files')
    os.mkdir(inp5+'/input_GDX')
    os.mkdir(inp5+'/output_GDX')
    iG.run_GAMs(inp0, inp1, inp2, inp3, inp4, inp5, parameter_flags = inp6)
print(len(inputs), end = ' total runs to be done ... ')
print('\nto check progress, look at number of run_N folders in GAMs/optimization_runs/')
with Pool(processes = 16) as pool:
    results = pool.starmap(mask_func, inputs)

2187 ... to check progress, look at most recent run_N folder in GAMs/optimization_runs/


In [ ]:
# intepret results to look for best values